In [1]:
import os
import math
import random
import operator
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import math

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from operator import itemgetter

In [2]:
def load_file(data_file_path):
    data_df = pd.read_csv(data_file_path, delimiter=";")
    return data_df


def compute_reward_individual(num_calls, result, cost_per_call = 1):
    if result == "yes":
        return (11 - (cost_per_call * num_calls))
    elif result == "no":
        return (-1 * (cost_per_call * num_calls))
    

def compute_y_position_label(y_val):
    if y_val >= 0.0:
        return 'bottom'
    else:
        return 'top'
    

def div(a,b):
    if int(b) == 0:
        return 0.0
    else:
        return a/b
        

def plot_graph(x, y, key, bar_plot, digregard_x_axis):
    style = dict(size=8, color='red')
    new_x = []
    if digregard_x_axis:
        for i in range(1, len(x)+1):
            new_x.append(i)
    else:
        new_x = x
    fig, ax = plt.subplots(figsize = (14, 6))
    # plt.xticks(np.arange(len(x)), x, rotation=75)
    # ax.tick_params(axis='x', which='major', pad=20)
    if bar_plot:
        ax.bar(new_x, y, 0.75)
    else:
        # plt.xlim((x[0],x[-1]))
        ax.plot(x, y)
    if digregard_x_axis:
        for i in range(1, len(x)+1):
            ax.text(i, y[i-1], x[i-1], ha='center', verticalalignment=compute_y_position_label(y[i-1]), **style)
        ax.axes.get_xaxis().set_visible(False)
    else:
        for i in range(1, len(x)+1):
            ax.text(x[i-1], y[i-1], y[i-1], ha='center', verticalalignment=compute_y_position_label(y[i-1]), **style)
    # Setting title and labels.
    title = "Expected reward vs " +  str(key)
    ax.set(xlabel=str(key), ylabel='Expected Reward', title=title)
    # plt.axvline(x=0, color ="black")
    plt.axhline(y=0, color ="black")
    # ax.grid()
    plt.tight_layout()
    plt.show()
    

def plot_graph_balance(x, y, key, step):
    style = dict(size=8, color='black')
    fig, ax = plt.subplots(figsize = (14, 6))
    # plt.xticks(np.arange(len(x)), x, rotation=75)
    # ax.tick_params(axis='x', which='major', pad=20)
    ax.bar(x, y, 0.75)
    # Setting title and labels.
    title = "Expected reward vs " +  str(key)
    ax.set(xlabel= "Average yearly balance", ylabel='Expected Reward', title=title)
    # plt.axvline(x=0, color ="black")
    plt.axhline(y=0, color ="black")
    # ax.grid()
    plt.tight_layout()
    tick_labels = ['-ve', '-ve', '-ve']
    for i in range (1, len(x)+1):
        tick_labels.append(str((i-1)*step) +'-' + str(i * step))
    ax.set_xticklabels(tick_labels ,rotation='vertical')
    plt.show()

In [3]:
def compute_plot_value_rows_by_key(mkt_df, key, bar_plot, digregard_x_axis):
    values = mkt_df[key].unique()
    reward_value ={}
    total_count = 0
    for value_ref in values:
        reward_value[value_ref] = {'value':0.0, 'count':0}
    for index, row in mkt_df.iterrows():
        value_ref = row[key]
        reward_value[value_ref]['value'] += (compute_reward_individual(row['campaign'], row['y']))
        reward_value[value_ref]['count'] += 1
        total_count += 1
    reward_value_list = reward_value.items()
    reward_value_list_sorted = sorted(reward_value_list, key=lambda tup: tup[1]['value'], reverse=True)
    x = []
    y = []
#     print("List of x,y pairs: ")
    i = 1
    for item in reward_value_list_sorted:
#         print(item, end='\t')
        if i%6 == 0:
            print("\n")
        x.append(item[0])
        y.append((item[1]['value']/item[1]['count']))
        i += 1
#     print("Y is:")
#     print(y)
    plot_graph(x, y, key, bar_plot, digregard_x_axis)

    
def compute_plot_balance_value(mkt_df, step = 2500):
    min = mkt_df['balance'].values.min()
    max = mkt_df['balance'].values.max()
    reward_value = {}
    total_count = 0
    for i in range(min, max, step):
        reward_value[(int(i/step))] = {'value':0.05, 'count':0}
#         print("Max is:", max)
    for index, row in mkt_df.iterrows():
        bal = row['balance']
        bal_index = None
        if bal <0.0:
            bal_index = -1
        else:
            bal_index = int(bal/step)
        reward_value[bal_index]['value'] += compute_reward_individual(row['campaign'], row['y'])
        reward_value[bal_index]['count'] += 1
        total_count += 1
    x = [str(key) for key in reward_value.keys()]
    y = [div(item['value'],item['count']) for item in reward_value.values()]
#     print(reward_value)
    plot_graph_balance(x, y, 'balance', step)

In [4]:
# This function takes the data corresponding to a feature set and computes the expected values for calls
# ranging from 1 .. 10.
def compute_expected_reward_feature_set(fs_df, no_calls):
    expected_values_calls = []
    for i in range(0, no_calls):
        expected_values_calls.append({'value':0.0, 'count':0, 'expected':0.0})
    for index, row in fs_df.iterrows():
        expected_values_calls[row['campaign']-1]['value'] += compute_reward_individual(row['campaign'], row['y'])
        expected_values_calls[row['campaign']-1]['count'] += 1
    for i in range(0, no_calls):
        expected_values_calls[i]['expected'] = div(expected_values_calls[i]['value'], expected_values_calls[i]['count'])
    return expected_values_calls

In [78]:
# Given a dictionary of what attributes comprise a feature set, we can get all rows corresponding to this feature set.
def extract_rows_feature_set(fs_df, feature_labels = {'education':['tertiary', 'secondary'], 'job':['technician']}):
    for key in feature_labels:
        feature_labels_query_str = ''
        arr = feature_labels[key]
        for label in arr:
            feature_labels_query_str += (key + ' == "'+ label + '" | ')
        feature_labels_query_str = feature_labels_query_str[:-3]
        fs_df = fs_df.query(feature_labels_query_str)
    return fs_df


In [82]:
current_dir = os.getcwd()
mkt_df = load_file(current_dir + '/bank-full.csv')
mkt_df_filtered = mkt_df[(mkt_df['campaign']>=1) & (mkt_df['campaign']<=10)] # & (mkt_df['education']!="unknown")
# df1 = mkt_df_filtered.head(300)
print(mkt_df_filtered.columns)
print("\n")
# This is working.
# result = extract_rows_feature_set(mkt_df_filtered)
# compute_plot_value_rows_by_key(mkt_df_filtered, 'age', True, True)
# compute_plot_value_rows_by_key(mkt_df_filtered, 'education', True, False)
# compute_plot_value_rows_by_key(mkt_df_filtered, 'marital', True, False)
# compute_plot_value_rows_by_key(mkt_df_filtered, 'job', True, False)
# compute_plot_value_rows_by_key(mkt_df_filtered, 'default', True, False)
# compute_plot_value_rows_by_key(mkt_df_filtered, 'housing', True, False)
# compute_plot_value_rows_by_key(mkt_df_filtered, 'loan', True, False)
# compute_plot_balance_value(mkt_df_filtered)

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')




In [8]:
# print(len(mkt_df_filtered[(mkt_df_filtered['education'] == 'secondary') & (mkt_df_filtered['y'] == 'yes')]))
# print(len(mkt_df_filtered[(mkt_df_filtered['education'] == 'secondary') & (mkt_df_filtered['y'] == 'no')]))
# total_v = 0.0
# for index, row in mkt_df_filtered.iterrows():
#     if row['education'] == 'secondary':
#         if row['y'] == "yes":
#             total_v += (11 - row['campaign'])
#         else:
#             total_v -= (row['campaign'])
# print(total_v)
# print()

In [23]:
# df1.to_excel("output2.xlsx")

In [37]:
#Important information about dataset.
print(len(mkt_df_filtered[mkt_df_filtered['y']=="yes"]))
print(len(mkt_df_filtered[mkt_df_filtered['y']=="no"]))

5242
38773


In [42]:
my_str =  'education == "tertiary"'
mkt_df_filtered.query(my_str)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
21,56,management,married,tertiary,no,779,yes,no,unknown,5,may,164,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45194,59,management,married,tertiary,no,138,yes,yes,cellular,16,nov,162,2,187,5,failure,no
45198,37,management,married,tertiary,no,1428,no,no,cellular,16,nov,333,2,-1,0,unknown,no
45201,53,management,married,tertiary,no,583,no,no,cellular,17,nov,226,1,184,4,success,yes
45203,23,student,single,tertiary,no,113,no,no,cellular,17,nov,266,1,-1,0,unknown,yes


In [84]:
mkt_df_filtered.query('age == 93')

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
44262,93,retired,married,unknown,no,775,no,no,cellular,22,jul,860,2,177,7,success,yes
44420,93,retired,married,unknown,no,775,no,no,cellular,4,aug,476,2,13,9,success,yes


In [85]:
mkt_df_filtered.query('age == 87')

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
41923,87,retired,divorced,primary,no,6746,no,no,telephone,21,oct,749,1,-1,0,unknown,yes
42081,87,retired,married,primary,no,230,no,no,cellular,30,oct,144,1,-1,0,unknown,yes
42494,87,retired,married,secondary,no,433,no,no,telephone,15,dec,150,1,-1,0,unknown,no
42635,87,retired,married,primary,no,2190,no,no,telephone,12,jan,512,2,-1,0,unknown,yes
